# Weight tracking, analysis and forecast

### Setup

In [6]:
import numpy as np
import csv
import datetime
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
import os
import sys
import pandas as pd
import seaborn as sns
import tensorflow as tf

from zipfile import ZipFile

In [2]:
print(f'Python = {sys.version}')
print(f'Numpy = {np.__version__}')
print(f'Tensorflow = {tf.__version__}')

Python = 3.8.11 (default, Aug  6 2021, 09:57:55) [MSC v.1916 64 bit (AMD64)]
Numpy = 1.20.3
Tensorflow = 2.5.0


### Data Extraction

=> file "data_BEN.zip" contains the file "weight.csv" with all daily measurements of total mass, fat mass.

=> file "Suivi_Poids.csv" has manually-entered daily numbers for total mass and fat mass, plus nutrition data (lipid, carbs, proteines), and calories data (ingested, spent with cardio, spent with strength training)

In [5]:
ROOT = os.getcwd() + '/'
print(f'current working directory is {ROOT}')
print(f'contains :')
print(f'{os.listdir(ROOT)}')

current working directory is C:\Users\benjamin.deporte\Documents\095_Code_Python_Toy\Weight/
contains :
['.git', '.ipynb_checkpoints', 'data_BEN.zip', 'desktop.ini', 'pat.txt', 'README.md', 'Suivi_Poids.csv', 'weight.ipynb']


In [40]:
class GetData():
    
    """
    Basic class to to all the data extraction work :
    - extracts the two files
    - replaces manually entered numbers for total mass and fat mass by calculated averages if available
    - provides method to export processed data
    """
    
    def __init__(self, zip_file_name, manual_file_name):
        self.zip_file_name = zip_file_name
        self.manual_follow_up_file_name = manual_file_name
    
    def extract(self):
        # extract data from the two files
        # first : data_BEN.zip
        
        root = os.getcwd() + '/'
        weight_file_path = root + 'tmp'

        with ZipFile(self.zip_file_name, 'r') as fichier_zip:
            print(f'Extraction fichier Zip Healthmate dans {weight_file_path}...')
            fichier_zip.extractall(path = weight_file_path)
            print(f'... Done')
        
        weight_file_csv_name = weight_file_path + '/weight.csv'
        fields = ['date', 
                  'MT', 
                  'MG']

        self.raw_from_zip = self.__extract_data(weight_file_csv_name, 
                                                fields,
                                                skip=1, 
                                                delimiter=',')  # gets a list of dictionnaries
        
        # second : Suivi_Poids.csv
        
        fields = ['date', 
                  'Masse Totale', 
                  'Masse Grasse', 
                  'Calories in', 
                  'Glucides', 'Lipides', 'Proteines', 
                  'Calories Exercice Brut', 'C_Ex_Cardio', 'C_Ex_Strength',
                  'Verif']
        
        self.raw_from_manual = self.__extract_data(self.manual_follow_up_file_name, 
                                                   fields,
                                                   skip=2, 
                                                   delimiter=';')  # returns and stores a list of disctionnaries
        
        # consolidates the two files in only one, averaging the daily measurements from the weight.csv in zip into
        # the daily number
        
        # first, get the raw_from_zip and creates a list of dictionnaries, with keys being date objects, 
        # and values all measurements of the day :
        # raw_daily = [ { 'date' : date_object1 , 'mt' : [list of total mass floats ] , 'mg' : [list of fat mass floats ] },
        #               { 'date' : date_object2 , 'mt' : [list of total mass floats ] , 'mg' : [list of fat mass floats ] },
        #               ... ]
        
        # first, find all the unique dates
        all_dates = [ self.__conv_to_date_num(current_dict.get('date')) for current_dict in self.raw_from_zip ] # list of date objects, same may be duplicate
        unique_dates = sorted(list(set(all_dates)))
        self.raw_daily = [ { 'date' : date, 'mt' : [], 'mg' : [] } for date in unique_dates ]
        # then iterate on all records, and populate for each unique date :
        for cdict in self.raw_from_zip:
            cdate = self.__conv_to_date_num(cdict.get('date'))
            cmt = self.__conv_to_float(cdict.get('MT'))
            cmg = self.__conv_to_float(cdict.get('MG'))
            for cdict2 in self.raw_daily:
                if cdict2.get('date') == cdate:
                    
            
            
    
    def __extract_data(self,
                       filename, 
                       champs,
                       skip=2,
                       delimiter=','):
        """
        Helper function that reads a csv file, returns a list of dictionnaries
        Each dict is a line, ie a daily measurement, in the file.
        Skips the first line(s)
        
        Parameters :
        filename (str) : name of the *.csv file
        fieldnames (sequence) : sequence of the columns names
        skip (int) : number of lines to skip at the beginning of the file
        delimiter (str) : character used as a delimiter
        """

        data = []
        with open(filename, newline='') as csvfile:
            fichier = csv.DictReader(csvfile, fieldnames=champs, delimiter=delimiter)
            for i in range(skip):  # skip <skip> lines at the beginning of the file
                next(fichier)
            for row in fichier:
                data.append(row)

        return data
    
    def __conv_to_date_str(self, date_string : str) -> datetime.date:
        """
        helper function that converts string outputs of dates, 
        with format "YY-month_name-day", and returns a date object
        from datetime.
        """

        dict_mois = {'août' : 8, 
                     'sept.' : 9,
                     'oct.' : 10,
                     'nov.' : 11,
                     'déc.' : 12,
                     'janv.' : 1,
                     'févr.' : 2,
                     'mars' : 3,
                     'avr.' : 4,
                     'mai' : 5,
                     'juin' : 6,
                     'juil.' : 7
                    }
        d = date_string.split(' ')[0]  # récupère la date en début de string : 2xxx-MM-DD
        d = d.split('-')  # récupère year, month, day

        # print(d)

        try:
            day = int(d[0])
        except ValueError:
            raise NameError('problème de format dans un champ date (jour)')

        try:
            year = 2000 + int(d[2])
        except ValueError:
            raise NameError('problème de format dans un champ date (année)')

        try:
            month = int(d[1])
        except ValueError:
            try:
                month = dict_mois.get(d[1])
            except ValueError:
                raise NameError('problème de format dans un champ date (mois)')

        date_object = datetime.date(year, month, day)

        return date_object
    
    def __conv_to_date_num(self, date_string : str) -> datetime.date:
        """
        helper function that converts a "YYYY-MM-DD" string into a date object
        """

        d = date_string.split(' ')[0]  # récupère la date en début de string : 2xxx-MM-DD
        d = d.split('-')  # récupère year, month, day

        # print(d)

        try:
            day = int(d[2])
        except ValueError:
            raise NameError('problème de format dans un champ date (jour)')

        try:
            year = int(d[0])
        except ValueError:
            raise NameError('problème de format dans un champ date (année)')

        try:
            month = int(d[1])
        except ValueError:
            raise NameError('problème de format dans un champ date (mois)')

        date_object = datetime.date(year, month, day)

        return date_object
    
    def __conv_to_float(self, float_string:str) -> float:
        """
        conversion basique+ en float.

        renvoie 0 si string vide ou remplie d'espaces, ou string = '-'.
        """

        if type(float_string) is None:
            return 0

        float_string = float_string.replace(" ","")
        if not float_string:
            return 0
        if float_string == "-":
            return 0

        try:
            float_string = float_string.replace(" ","")
            valeur = float(float_string.replace(',','.'))
        except ValueError:
            raise NameError('une tentative de conversion en float a échouée car string non compatible')

        return valeur

In [41]:
obj = GetData(ROOT + 'data_BEN.zip', ROOT + 'Suivi_Poids.csv')

In [42]:
obj.extract()

Extraction fichier Zip Healthmate dans C:\Users\benjamin.deporte\Documents\095_Code_Python_Toy\Weight/tmp...
... Done
[datetime.date(2021, 10, 24), datetime.date(2021, 10, 24), datetime.date(2021, 10, 24), datetime.date(2021, 10, 24), datetime.date(2021, 10, 24), datetime.date(2021, 10, 24), datetime.date(2021, 10, 23), datetime.date(2021, 10, 23), datetime.date(2021, 10, 23), datetime.date(2021, 10, 23), datetime.date(2021, 10, 23), datetime.date(2021, 10, 22), datetime.date(2021, 10, 22), datetime.date(2021, 10, 22), datetime.date(2021, 10, 22), datetime.date(2021, 10, 22), datetime.date(2021, 10, 22), datetime.date(2021, 10, 22), datetime.date(2021, 10, 22), datetime.date(2021, 10, 21), datetime.date(2021, 10, 21), datetime.date(2021, 10, 21), datetime.date(2021, 10, 21), datetime.date(2021, 10, 21), datetime.date(2021, 10, 21), datetime.date(2021, 10, 21), datetime.date(2021, 10, 21), datetime.date(2021, 10, 20), datetime.date(2021, 10, 20), datetime.date(2021, 10, 20), datetime.da

In [32]:
print(obj.raw_from_zip)

[{'date': '2021-10-24 05:42:13', 'MT': '67.05', 'MG': '8.09', None: ['', '', '', '']}, {'date': '2021-10-24 05:41:38', 'MT': '67.10', 'MG': '8.15', None: ['', '', '', '']}, {'date': '2021-10-24 05:41:05', 'MT': '67.10', 'MG': '8.10', None: ['', '', '', '']}, {'date': '2021-10-24 05:40:27', 'MT': '67.10', 'MG': '8.15', None: ['', '', '', '']}, {'date': '2021-10-24 05:39:49', 'MT': '67.05', 'MG': '8.18', None: ['', '', '', '']}, {'date': '2021-10-24 05:39:09', 'MT': '67.05', 'MG': '8.18', None: ['', '', '', '']}, {'date': '2021-10-23 08:50:48', 'MT': '66.55', 'MG': '7.74', None: ['', '', '', '']}, {'date': '2021-10-23 08:50:11', 'MT': '66.50', 'MG': '7.73', None: ['', '', '', '']}, {'date': '2021-10-23 08:49:31', 'MT': '66.50', 'MG': '7.82', None: ['', '', '', '']}, {'date': '2021-10-23 08:48:51', 'MT': '66.50', 'MG': '7.83', None: ['', '', '', '']}, {'date': '2021-10-23 08:48:08', 'MT': '66.60', 'MG': '7.85', None: ['', '', '', '']}, {'date': '2021-10-22 06:15:14', 'MT': '67.20', 'MG': 

In [33]:
print(obj.raw_from_manual)

[{'date': '1-août-20', 'Masse Totale': ' 107,2   ', 'Masse Grasse': '', 'Calories in': '', 'Glucides': ' -     ', 'Lipides': ' -     ', 'Proteines': ' -     ', 'Calories Exercice Brut': ' -     ', 'C_Ex_Cardio': ' -     ', 'C_Ex_Strength': ' -     ', 'Verif': ' -     ', None: ['', '']}, {'date': '2-août-20', 'Masse Totale': ' 106,8   ', 'Masse Grasse': '', 'Calories in': '', 'Glucides': ' -     ', 'Lipides': ' -     ', 'Proteines': ' -     ', 'Calories Exercice Brut': ' -     ', 'C_Ex_Cardio': ' -     ', 'C_Ex_Strength': ' -     ', 'Verif': ' -     ', None: ['', '']}, {'date': '3-août-20', 'Masse Totale': ' 107,2   ', 'Masse Grasse': '', 'Calories in': ' 1 918   ', 'Glucides': ' 195   ', 'Lipides': ' 89   ', 'Proteines': ' 72   ', 'Calories Exercice Brut': ' -     ', 'C_Ex_Cardio': ' -     ', 'C_Ex_Strength': ' -     ', 'Verif': ' -     ', None: ['', '']}, {'date': '5-août-20', 'Masse Totale': ' 105,5   ', 'Masse Grasse': '', 'Calories in': ' 1 949   ', 'Glucides': ' 213   ', 'Lipides'

# Extraction data brute des CSVs

### Traite deux fichiers :

=> un fichier "data_BEN.zip" avec les datas issus de HealthMate :
  - masse totale
  - masse grasse
  - plusieurs pesées à la suite (5 mini) avec le même protocole (matin a jeun après toilettes) 
 
=> un fichier "Suivi_Poids.csv" avec les datas rentrées à la main, et notamment les données d'exercice : 
  - masse totale, masse grasse (moyennes des données balance calculées et rentrées à la main)
  - données MyFitnessPal : calories in, macros (glucides, lipides, proteines)
  - données exercice issues des enregistrements ceinture Polar (calories_exercies, calories_cardio, calories_muscu)

In [ ]:
RACINE = os.getcwd() + '/'

print(os.getcwd())

### Extraction fichier HealthMate : données quotidiennes multiples, dans weight.csv dans l'archive Zip 'data_BEN.zip'

In [ ]:
ZIPFILE_NAME = RACINE + 'data_BEN.zip'
WEIGHT_FILE_PATH = RACINE + 'tmp'

from zipfile import ZipFile

with ZipFile(ZIPFILE_NAME, 'r') as fichier_zip:
    print(f'Extraction fichier Zip Healthmate dans {WEIGHT_FILE_PATH}...')
    fichier_zip.extractall(path = WEIGHT_FILE_PATH)
    print(f'... Done')

In [ ]:
WEIGHT_FILE_CSV_NAME = WEIGHT_FILE_PATH + '/weight.csv'
CHAMPS = ['date', 'MT', 'MG']

data_brute = extrait_data(WEIGHT_FILE_CSV_NAME, CHAMPS, skip=1, delimiter=',')  # récupère une liste de dictionnaires

### Extraction données complètes par jour avec calories, macros, etc. depuis 'Suivi_Poids.csv'

In [ ]:
FILE_CALS_EX_CSV_NAME = RACINE + "Suivi_Poids.csv"
CHAMPS = ['date', 'Masse Totale', 'Masse Grasse', 'Calories in', 'Glucides', 'Lipides', 'Proteines', 'Calories Exercice Brut', 'C_Ex_Cardio', 'C_Ex_Strength','Verif']
# NB : CHAMPS doit impérativement contenir un champ 'date'

In [ ]:
poids_cal_exos = extrait_data(FILE_CALS_EX_CSV_NAME, CHAMPS, skip=2, delimiter=';')  # récupère une liste de dictionnaires

In [ ]:
poids_cal_exos

**0- Fonctions helper pour conversions**

In [ ]:
def conv_to_date_str(date_string : str) -> datetime.date:
    """
    fonction qui convertit les string sorties de l'extraction en date,
    au format "YY-month_name-day",
    retourne un objet date de la library datetime.
    
    Permet de faire des calculs sur les dates ensuite.
    """
    
    dict_mois = { 'août' : 8, 
                 'sept.' : 9,
                 'oct.' : 10,
                 'nov.' : 11,
                 'déc.' : 12,
                 'janv.' : 1,
                 'févr.' : 2,
                 'mars' : 3,
                 'avr.' : 4,
                 'mai' : 5,
                 'juin' : 6,
                 'juil.' : 7
                }
    d = date_string.split(' ')[0]  # récupère la date en début de string : 2xxx-MM-DD
    d = d.split('-')  # récupère year, month, day
    
    # print(d)
    
    try:
        day = int(d[0])
    except ValueError:
        raise NameError('problème de format dans un champ date (jour)')
        
    try:
        year = 2000 + int(d[2])
    except ValueError:
        raise NameError('problème de format dans un champ date (année)')
    
    try:
        month = int(d[1])
    except ValueError:
        try:
            month = dict_mois.get(d[1])
        except ValueError:
            raise NameError('problème de format dans un champ date (mois)')
        
    date_object = datetime.date(year, month, day)
    
    return date_object

In [ ]:
def conv_to_date_num(date_string : str) -> datetime.date:
    """
    fonction qui convertit les string sorties de l'extraction en date,
    au format "YYYY-MM-DD",
    retourne un objet date de la library datetime.
    
    Permet de faire des calculs sur les dates ensuite.
    """
    
    d = date_string.split(' ')[0]  # récupère la date en début de string : 2xxx-MM-DD
    d = d.split('-')  # récupère year, month, day
    
    # print(d)
    
    try:
        day = int(d[2])
    except ValueError:
        raise NameError('problème de format dans un champ date (jour)')
        
    try:
        year = int(d[0])
    except ValueError:
        raise NameError('problème de format dans un champ date (année)')
    
    try:
        month = int(d[1])
    except ValueError:
        raise NameError('problème de format dans un champ date (mois)')
        
    date_object = datetime.date(year, month, day)
    
    return date_object

In [ ]:
def conv_to_float(float_string:str) -> float:
    """
    conversion basique+ en float.
    
    renvoie 0 si string vide ou remplie d'espaces, ou string = '-'.
    """
    
    if type(float_string) is None:
        return 0
    
    float_string = float_string.replace(" ","")
    if not float_string:
        return 0
    if float_string == "-":
        return 0
    
    try:
        float_string = float_string.replace(" ","")
        valeur = float(float_string.replace(',','.'))
    except ValueError:
        raise NameError('une tentative de conversion en float a échouée car string non compatible')
        
    return valeur


# 1- Analyse de $E(Y|X=x)$ et $Var(Y|X=x)$ à partir du fichier complet


In [ ]:
data_brute

In [ ]:
# construit la liste des dates uniques dans data_brute

CUT_OFF_DATE = datetime.date(2020,8,1)  # on ne prend les valeurs qu'à partir du 1er Août 2020 !

liste_dates = [ conv_to_date_num(current_dict.get('date')) for current_dict in data_brute if conv_to_date_num(current_dict.get('date')) >= CUT_OFF_DATE ]
liste_dates = list(sorted(set(liste_dates)))
  

In [ ]:
# construit les dictionnaires des valeurs de MT et MG :
# - keys = valeur de datetime.date (date object)
# - values = liste des valeurs (float) du jour

dict_mt = dict( (d, []) for d in liste_dates )  # initialise le dict MT avec des listes vides des valeurs du jour
dict_mg = dict( (d, []) for d in liste_dates )  # idem pour MG

for element in data_brute:
    if conv_to_date_num(element.get('date')) >= CUT_OFF_DATE:  # nettoyage des data : on ne prend qu'après le cut_off_date
        date_object = conv_to_date_num(element.get('date'))
        mt = conv_to_float(element.get('MT'))
        dict_mt[date_object].append(mt)
        mg = conv_to_float(element.get('MG'))
        dict_mg[date_object].append(mg)
        

In [ ]:
dict_mg

In [ ]:
# construit les dictionnaires d'écart type pour MT et MG
# - keys = valeur de datetime.date (date object)
# - values = écart-type (float) si calculé > 0

std_mt = dict( (d, np.std(vals)) for d,vals in dict_mt.items() if np.std(vals)>0 )
std_mg = dict( (d, np.std(vals)) for d,vals in dict_mg.items() if np.std(vals)>0 )

In [ ]:
std_mt_for_bplot = [ v for v in std_mt.values() ]
std_mg_for_bplot = [ v for v in std_mg.values() ]

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16, 8))

ax[0].boxplot(std_mt_for_bplot, notch=True)
ax[0].set_title('Ecart type Masse Totale')
ax[1].boxplot(std_mg_for_bplot, notch=True)
ax[1].set_title('Ecart type Masse Grasse')
plt.show()

PREC = 3

print(f'Mediane StdDev Masse Totale = {np.around(np.median(std_mt_for_bplot),decimals=PREC)}, \
    Moyenne StdDev Masse Totale = {np.around(np.mean(std_mt_for_bplot), decimals=PREC)}')
print(f'Mediane StdDev Masse Grasse = {np.around(np.median(std_mg_for_bplot),decimals=PREC)}, \
    Moyenne StdDev Masse Grasse = {np.around(np.mean(std_mg_for_bplot), decimals=PREC)}')

In [ ]:
med_std_dev_mg = np.around(np.median(std_mg_for_bplot),decimals=PREC)   # mediane de l'écart-type sur la mesure de la MG
last_mt = np.mean(dict_mt.get(list(dict_mt.keys())[-1]))   # moyenne de la dernière valeur de la masse totale mesurée

print(f'Intervalle de confiance à 95% sur mesure de body fat% (+/- 2 sigma): +/- {np.around(2*med_std_dev_mg/last_mt*100, decimals=PREC)}%')

# 2- Calcul du body fat% : $E(MG/MT)$ ou $E(MT)/E(MG)$ ?

In [ ]:
# calcul de E(MG/MT) et E(MT) / E(MG)
e_mg_mt = []
std_dev_mg_mt = []  # là on calcule l'écart type des valeurs de mg/mt du jour
e_mg_e_mt = []

for mt_data, mg_data in zip(dict_mt.items(), dict_mg.items()):
    # vérifie qu'on est bien sur la même date !
    if mt_data[0] != mg_data[0]:
        NameError('dates différentes dans dict_mt et dict_mg, impossible calculer MG/MT du jour')
    mt = np.array(mt_data[1])
    mg = np.array(mg_data[1])
    
    # vérifie qu'il y a bien le même nombre de valeurs dans la journée
    if len(mt) != len(mg):
        NameError('nombre de valeurs différentes dans une même journée dans dict_mt et dict_mg')
        
    if np.sum(np.abs(mg)) > 0 and np.sum(np.abs(mt)) > 0:
        e_mg_mt.append(np.mean(mg/mt))
        std_dev_mg_mt.append(np.std(mg/mt))  # calcule l'écart type de la liste des calculs mg/mt du jour
        e_mg_e_mt.append(np.mean(mg)/np.mean(mt))
        
e_mg_mt = np.array(e_mg_mt)
e_mg_e_mt = np.array(e_mg_e_mt)
std_dev_mg_mt = np.array(std_dev_mg_mt)

diff = np.abs(e_mg_mt - e_mg_e_mt)       

In [ ]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(nrows = 1, ncols = 4, figsize=(24,8) )

ax1.plot(e_mg_mt)
ax1.grid()
ax1.set_title('Body Fat calcul 1 : Moyenne de (MG sur MT)')

ax2.plot(e_mg_e_mt)
ax2.grid()
ax2.set_title('Body Fat calcul 2 : Moyenne de MG sur Moyenne de MT')

ax3.plot(diff)
ax3.grid()
ax3.set_title('Ecart entre E(MG/MT) et E(MG)/E(MT)')

ax4.plot(std_dev_mg_mt * 100)
ax4.grid()
ax4.set_title('Ecart type des valeurs MG/MT du jour, en %')

print(f'Erreur maximale commise entre E(MT/MG) et E(MT)/E(MG) = {np.around(max(diff), decimals=8)}')

print(f'Moyenne Ecart type de la mesure de masse grasse (en %, comparable à la valeur MG/MT) : {np.around(np.mean(std_dev_mg_mt * 100), 2)}%')

# 2- Collection des données de calories, calcul MG%, etc

### On commence par créer un numpy array avec toutes les datas :
- date ordinale (convertie des datetime objects)
- masse totale
- masse grasse
- pourcentage de masse grasse (si masse totale et masse grasse > 0)
- calories ingérées
- proteines
- glucides
- lipides
- calories exercice (éventuellement 0)
- calories cardio (éventuellement 0)
- calories muscu (éventuellement 0)

In [ ]:
X = []
keys = list(poids_cal_exos[0].keys())

In [ ]:
today = datetime.date.today()

for d in poids_cal_exos:
    
    date_d = conv_to_date_str(d.get('date'))
    
    if date_d <= today:
        X_day =[ conv_to_float(d.get(k)) if k != 'date' else date_d for k in keys ]
        
        mt = conv_to_float(d.get('Masse Totale'))
        mg = conv_to_float(d.get('Masse Grasse'))
        if mt>0 and mg>0:
            bf = mg/mt
        else:
            bf = 0.0
            
        X_day.append(bf)  # rajoute body fat% en dernière valeur
        # print(X_day)
        X.append(X_day)
        
keys.append('Body Fat')

In [ ]:
X = np.array(X)
X.shape

In [ ]:
X[0]

In [ ]:
# on rajoute la date ordinale en avant-dernière colonne
keys[keys.index('Verif')]='date_ordinale'
X[:,keys.index('date_ordinale')] = [x.toordinal() for x in X[:,keys.index('date')] ]

In [ ]:
keys

In [ ]:
X

In [ ]:
print(f'{len(X)} data depuis {CUT_OFF_DATE} jusqu"à {today}')

# 3- Affichage basique

In [ ]:
def basic_plot(dates, y, 
               grid=True,
               title='titre', perc=False,
               moyenne_glissante=False, n_moy=7, 
               reg_lineaire=False, n_reg=30):
    
    """Affiche y suivant les dates, avec grid ou pas, avec moyenne glissante et régression linéaire ou pas.

    Args:
        dates ([datetime objects]): [dates au format datetime]
        y ([float]): [la valeur à afficher]
        grid (bool, optional): [présence ou pas de la grille]. Defaults to True.
        title(string, optional): [titre]. Defaults to 'titre'
        moyenne_glissante (bool, optional): [affiche ou pas la moyenne glissante]. Defaults to False.
        nj_moyenne (int, optional): [fenêtre de calcul de la moyenne glissante]. Defaults to 7.
        reg_lineaire (bool, optional): [affiche ou pas la régression linéaire]. Defaults to False.
        nj_regression (int, optional): [fenêtre de calcul de la régression linéaire]. Defaults to 30.
    """
    
    fig, (ax1,ax2) = plt.subplots(nrows=1, ncols=2, figsize=(16,8))
    ax1.plot(dates, y, color='blue', marker='x')
    ax1.set_title(title)
    
    if grid: 
        ax1.grid()
    
    if moyenne_glissante:
        # plot moyenne glissante sur les n_moy dernières valeurs
        moy = np.zeros(shape=len(y))
        moy[0] = y[0]
        for i in range(1,len(y)):
            id = max(0, i-n_moy+1)
            moy[i] = np.mean(y[id:i])
        ax1.plot(dates, moy, color='red', marker='o')
    
    if reg_lineaire:
        # plot regression linéaire sur les n_reg dernières valeurs de y
        dates_ordinales = np.array([ d.toordinal() for d in dates]).reshape(-1,1)
        lr_model = LinearRegression().fit(dates_ordinales[-n_reg:], y[-n_reg:])   # fit sur les n_reg dernières valeurs
        reg_pred = lr_model.predict(dates_ordinales)
        ax1.plot(dates, reg_pred, color='green', marker='+')
        pente =lr_model.coef_[0] * 30 # coefficient par mois
        if perc==True: pente *= 100
        coeff = lr_model.score(dates_ordinales[-n_reg:], y[-n_reg:])
        ax2.text(0.1,0.8,f'régression calculée de {dates[-n_reg]} à {dates[-1]}')
        if perc==True:
            str_pente = f'pente = {np.around(pente, decimals=PREC)}% / mois'
        else:
            str_pente = f'pente = {np.around(pente, decimals=PREC)} / mois'
        ax2.text(0.1,0.7,str_pente)
        ax2.text(0.1,0.6,f'coefficient régression = {np.around(coeff * 100,1)}%')
        
        
    return fig

In [ ]:
# Récupère les data entre deux dates

start_date = datetime.date(2021,8,1)
start_date_index = list(X[:,0]).index(start_date)

end_date = datetime.date(2021,9,30)
end_date_index = list(X[:,0]).index(end_date)

X_trim = X[start_date_index:end_date_index]

In [ ]:
print(keys)

In [ ]:
# affiche certaines data depuis cette start date

data_of_interest = ['Masse Totale', 'Masse Grasse', 'Calories in', 'Body Fat']
idx = [ keys.index(l) for l in data_of_interest ]

In [ ]:
dates = [ el[0] for el in X_trim]
for id, l in zip(idx, data_of_interest):
    y = [el[id] for el in X_trim ]
    f = basic_plot(dates, y, grid=True, perc=(l=='Body Fat'), moyenne_glissante=True, reg_lineaire=True, title=l)   

# 4 - Heatmap corrélations

https://heartbeat.fritz.ai/seaborn-heatmaps-13-ways-to-customize-correlation-matrix-visualizations-f1c49c816f07

In [ ]:
start_date = datetime.date(2021,1,1)   # depuis le 1er Janvier 2021
start_date_index = list(X[:,0]).index(start_date)

end_date = datetime.date(2021,9,20)  # <== A CHANGER ICI
end_date_index = list(X[:,0]).index(end_date)

X_trim = X[start_date_index:end_date_index,1:]
df = pd.DataFrame(X_trim, columns=keys[1:]).astype(float)   # on créé une dataframe avec tous les floats (exclus les datetime.objects)

In [ ]:
df.head(20)

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
print(df.corr())

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

sns.heatmap(df.corr(), 
            annot=True, 
            fmt='.2g',
            vmin=-1, vmax=1, center=0,
            cmap='Greys')

## Commentaires sur le body fat% :
- corrélations évidentes du bf% avec Masse Totale, Masse Grasse, et Date
- corrélations avec Protéines (-.49), Lipides (0.28), C_ex_cardio (.19)

# Le temps des prévisions

On va chercher Masse_Totale(j+1), Masse_Grasse(j+1) = f(Masse_Totale(j), Masse_Grasse(j), C_in(j), Glu(j), Lip(j), Pro(j), C_cardio(j), C_strength(j))

Variables d'intérêt : on va chercher à prédire la **Masse Totale** et la **Masse Grasse**.

Les modèles sont donc des modèles de régression, avec une sortie y à deux dimensions.

La loss utilisée sur les vecteurs 2D sera la **norme 1** (mean absolute error), avec un poids différent entre la première coordonnée (qui varie typiquement entre 108 et 68kg sur le dataset), et la deuxième coordonnée (qui varie entre .33 et .12)

Les modèles regardés seront :
- prédiction näive (prédiction à J+1 = valeur de la veille)
- régression linéaire (NB : on cherchera la fenêtre temporelle pertinente, cad quels sont les N dernières valeurs à considérer pour avoir un bon niveau de précision)
- MLP
- LSTM

### Creation du dataset

On prend le X en entrée, et on construit un tf.Dataset avec comme y_true les valeurs de Masse Totale et Masse Grasse du jour suivant

In [ ]:
def create_dataset(X, 
                   start_date = datetime.date(2020,8,1), 
                   end_date = datetime.date.today()-datetime.timedelta(days=1),
                   # keys = ['date', 'Masse Totale', 'Masse Grasse', 'Calories in', 'Glucides', 'Lipides', 'Proteines', 'Calories Exercice Brut', 'C_Ex_Cardio', 'C_Ex_Strength', 'date_ordinale', 'Body Fat']
                   keys_to_keep = ['Masse Totale', 'Masse Grasse', 'Calories in', 'Glucides', 'Lipides', 'Proteines', 'C_Ex_Cardio', 'C_Ex_Strength']
                   ):

    """Construit et retourne un tf.dataset, à partir du fichier de données, entre les dates fournies

    Args:
        X ([np.array]): [c'est le tableau de données journalières :
        keys ([list]) : la liste des champs, defaults to ['Masse Totale', 'Masse Grasse', 'Calories in', 'Glucides', 'Lipides', 'Proteines', 'C_Ex_Cardio', 'C_Ex_Strength'] ]
        start_date ([datetime object], optional): [description]. Defaults to datetime.date(2020,8,1).
        end_date ([datetime objectf], optional): [description]. Defaults to datetime.today()-1.

    Returns : tf.Data.dataset
    """
    
    start_date_index = list(X[:,0]).index(start_date)
    end_date_index = list(X[:,0]).index(end_date)
    
    index_mt = keys.index('Masse Totale')
    index_mg = keys.index('Masse Grasse')
    
    idx_to_keep = [ keys.index(k) for k in keys_to_keep ]
                       
    X_trim = np.array(X[start_date_index:end_date_index-1,idx_to_keep],dtype=float)  # on ne prend pas la date sous format object, et on s'arrête à l'avant-dernière valeur
    
    y = np.array([ [X[i+1][index_mt],X[i+1][index_mg]] for i in range(start_date_index, end_date_index-1)], dtype=float)
    
    print(X_trim.shape)
    print(y.shape)
    
    return tf.data.Dataset.from_tensor_slices((X_trim, y))
    
    # return tf.data.Dataset.from_tensor_slices([X_trim, y])

In [ ]:
start_date = datetime.date(2020,9,1) 
end_date = datetime.date.today()-datetime.timedelta(days=1)

ds = create_dataset(X, start_date = start_date, end_date=end_date)

In [ ]:
ds

In [ ]:
for el in ds.take(3):
    print(el)

# Naive Forecasting

Ici, le modèle prédit pour le jour suivant, la valeur du jour : cela servira de baseline

In [ ]:
start_date = datetime.date(2020,9,1) 
end_date = datetime.date.today()-datetime.timedelta(days=1)

ds = create_dataset(X, start_date = start_date, end_date=end_date)

index_mt = 0
index_mg = 1
 
y_true = np.array([ el[1] for el in ds ], dtype = float )
y_pred = np.array([ [el[0][index_mt], el[0][index_mg]] for el in ds], dtype=float)

In [ ]:
y_true.shape

In [ ]:
y_pred.shape

In [ ]:
ds

In [ ]:
y_pred

In [ ]:
y_true

In [ ]:
def output_results(y_pred, y_true,
                   history = None,
                   start_date = datetime.date(2020,9,1),
                   end_date = datetime.date.today() - datetime.timedelta(days=1),
                   ):
    
    x = [ start_date + datetime.timedelta(days=i) for i in range(end_date.toordinal() - start_date.toordinal() - 1)]
    
    fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(32,12))

    ax[0][0].set_title("prediction masse totale")
    ax[0][0].plot(x, y_true[:,0], marker = 'x', label='true')
    ax[0][0].plot(x, y_pred[:,0], marker = '.', label='pred')
    ax[0][0].legend()
    ax[0][0].grid()

    ax[1][0].set_title("prediction masse grasse")
    ax[1][0].plot(x, y_true[:,1], marker = 'x', label='true')
    ax[1][0].plot(x, y_pred[:,1], marker = '.', label='pred')
    ax[1][0].legend()
    ax[1][0].grid()

    mae_mt = np.array( [np.abs([y[0][0] - y[1][0]]) for y in zip (y_true, y_pred) ] )
    mae_mt_moyenne = np.full( shape=len(mae_mt), fill_value = mae_mt.mean() )

    ax[0][1].set_title("erreur norme 1 prediction masse totale")
    ax[0][1].plot(x, mae_mt, marker = 'x')
    ax[0][1].plot(x, mae_mt_moyenne, marker = '.', label = 'moyenne')
    ax[0][1].legend()
    ax[0][1].grid()

    mae_mg = np.array( [np.abs([y[0][1] - y[1][1]]) for y in zip (y_true, y_pred) ] )
    mae_mg_moyenne = np.full( shape=len(mae_mg), fill_value = mae_mg.mean() )

    ax[1][1].set_title("erreur norme 1 prediction masse grasse")
    ax[1][1].plot(x, mae_mg, marker = 'x')
    ax[1][1].plot(x, mae_mg_moyenne, marker = '.', label = 'moyenne')
    ax[1][1].legend()
    ax[1][1].grid()
    
    if history:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize = (8,8))
        erreur = list(history.history.keys())[-1]
        values = history.history.get(erreur)
        ax.plot(values)
        ax.grid()
        norm_1 = np.array(values).mean()
        ax.set_title(erreur + ' vs Epoch')
        print(erreur + f' cumulée MT et MG sur validation set: {np.around(norm_1, decimals=3)} kg')
        
    window_last_days = 30
    
    print(f'Erreur moyenne norme 1 sur MT sur les {window_last_days} derniers jours : {np.around(mae_mt[-window_last_days:].mean(), decimals=3)} kg')
    print(f'Erreur moyenne norme 1 sur MG sur les {window_last_days} derniers jours : {np.around(mae_mg[-window_last_days:].mean(), decimals=3)} kg')

In [ ]:
output_results(y_pred, y_true, start_date=start_date, end_date=end_date)

# Basic MLP Model

In [ ]:
def custom_mae(y_pred, y_true):
    """Custom loss pour prendre en compte le delta entre MT et MG

    Args:
        y_pred ([float]): prédictions, (batch_size, 2)
        y_true ([float]): ground truth, (batch_size, 2)
        
    Returns : float
    """
    # calculating squared difference between target and predicted values 
    loss = tf.keras.backend.abs(y_pred - y_true)  # (batch_size, 2)
    
    # multiplying the values with weights along batch dimension
    loss = loss * [0.1, 0.9]          # (batch_size, 2)
                
    # summing both loss values along batch dimension 
    loss = tf.keras.backend.sum(loss, axis=1)        # (batch_size,)
    
    return loss

In [ ]:
start_date = datetime.date(2020,9,1) 
end_date = datetime.date.today()-datetime.timedelta(days=1)

ds = create_dataset(X, start_date = start_date, end_date=end_date)

In [ ]:
total_size = len(ds)
SPLIT_RATIO = 0.9
train_size = int(total_size * SPLIT_RATIO)

In [ ]:
ds_train = ds.take(train_size).batch(1)  # NB : le .batch() est nécessaire... pourquoi ?
ds_val = ds.skip(train_size).batch(1)  # NB : on valide sur les derniers xx %

In [ ]:
model_lr = tf.keras.models.Sequential([
    tf.keras.layers.Dense(units=64, activation='relu', name='couche_MLP_1', input_shape=([8])),
    tf.keras.layers.Dense(units=64, activation='relu', name='couche_MLP_2'),
    tf.keras.layers.Dense(units=64, activation='relu', name='couche_MLP_3'),
    tf.keras.layers.Dense(units=2,
                          name='output',
                          ),
    # tf.keras.layers.Reshape((2,1))
], name = 'basic_MLP')

tf.keras.utils.plot_model(model_lr,
                          show_shapes=True,
                          show_dtype=True,
                          show_layer_names=True)

In [ ]:
model_lr.compile(
    optimizer = 'adam',
    loss = 'mae',
    metrics = ['mae']
)

In [ ]:
cb = tf.keras.callbacks.EarlyStopping(
    monitor='loss',
    patience = 10,
    min_delta = 0,
    restore_best_weights=True
    )

In [ ]:
history = model_lr.fit(
    ds_train,
    epochs = 1000,
    callbacks=[cb],
    validation_data=ds_val
)

In [ ]:
y_true = np.array([ el[1] for el in ds ], dtype = float )
# y_pred = np.array([ [el[0][index_mt], el[0][index_bf]] for el in ds], dtype=float)

In [ ]:
y_true

In [ ]:
ds_total = ds.batch(1)
y_pred = model_lr.predict(ds_total)

In [ ]:
y_pred

In [ ]:
list(history.history.keys())[-1]

In [ ]:
output_results(y_pred, y_true, history=history, start_date=start_date, end_date=end_date)

# Approche par time-series

La prédiction au jour J du vecteur (masse totale, masse grasse), dépend de l'historique des données journalières sur une certaine période (à définir)